In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px

from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px

from scipy import stats
from pmdarima import auto_arima
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [14]:
df = pd.read_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Transformed_Onion_Prices.xlsx',index_col='Date')

In [22]:
df.index = pd.to_datetime(df.index, format='%m/%Y')

In [24]:
df

,W.sale Avg.Price(Rupees per Quintal),Retail Avg.Price (Rupees per Quintal)
Date,,
2000-01-01,455,553
2000-02-01,358,421
2000-03-01,350,430
2000-04-01,367,435
2000-05-01,369,435
...,...,...
2024-08-01,4102,5836
2024-09-01,4873,6446
2024-10-01,4862,6522


In [18]:
# Check NaN counts for all columns
nan_counts = df.isna().sum()
print("NaN counts per column:")
print(nan_counts)

# Iterate through the columns and find indices of NaN values
for column in df.columns:
    if df[column].isnull().any():  # Check if the column has any NaNs
        nan_indices = df[df[column].isnull()].index.tolist()
        print(f"\nNaN values found in column '{column}' at indices: {nan_indices}")

NaN counts per column:
W.sale Avg.Price(Rupees per Quintal)     0
Retail Avg.Price (Rupees per Quintal)    0
dtype: int64


In [42]:
zero_counts = (df == 0).sum()
print("Zero counts per column:")
print(zero_counts)

print("\nIndices of zero values per column:")
if (df['W.sale Avg.Price(Rupees per Quintal)'] == 0).any():  # Check if the column has any zeros
    zero_indices = df[df['W.sale Avg.Price(Rupees per Quintal)'] == 0].index.tolist()
    print(f"Zero values found in column 'W.sale Avg.Price(Rupees per Quintal)' at indices: {zero_indices}")
else:
    print(f"No zero values found in column 'W.sale Avg.Price(Rupees per Quintal)'.")

Zero counts per column:
W.sale Avg.Price(Rupees per Quintal)      0
Retail Avg.Price (Rupees per Quintal)    74
dtype: int64

Indices of zero values per column:
No zero values found in column 'W.sale Avg.Price(Rupees per Quintal)'.


In [36]:
# Define the column name
col = 'W.sale Avg.Price(Rupees per Quintal)'

# Function to calculate the average of same month in previous years with min_periods
def calculate_month_average_with_min_periods(row_date, data_series, min_periods=1):
    target_month = row_date.month
    previous_years_data = []
    
    for year_offset in range(1, 6):
        previous_year = row_date.year - year_offset
        try:
            previous_date = pd.Timestamp(year=previous_year, month=target_month, day=1)
            if previous_date in data_series.index and not pd.isna(data_series[previous_date]) and data_series[previous_date] != 0:
                previous_years_data.append(data_series[previous_date])
        except:
            continue
    
    # Calculate mean only if number of data points >= min_periods
    if len(previous_years_data) >= min_periods:
        return np.mean(previous_years_data)
    else:
        return np.nan

# Replace zeros with NaN
df[col] = df[col].replace(0, np.nan)

# Create a copy of the column with NaN values
filled_values = df[col].copy()

# For each NaN value, calculate the replacement
for date in df[df[col].isna()].index:
    filled_values[date] = calculate_month_average_with_min_periods(date, df[col], min_periods=1)

# Update the column with the filled values
df[col] = filled_values

# If there are still NaN values (no relevant history available), use overall mean
if df[col].isna().any():
    overall_mean = df[col].mean()
    df[col] = df[col].fillna(overall_mean)
    print(f"Some values filled with overall mean: {overall_mean}")

# Display the first few rows to verify


Date
2000-01-01    455.0
2000-02-01    358.0
2000-03-01    350.0
2000-04-01    367.0
2000-05-01    369.0
Name: W.sale Avg.Price(Rupees per Quintal), dtype: float64

In [50]:
df

,Date,W.sale Avg.Price(Rupees per Quintal)
0,2000-01-01,455.0
1,2000-02-01,358.0
2,2000-03-01,350.0
3,2000-04-01,367.0
4,2000-05-01,369.0
...,...,...
295,2024-08-01,4102.0
296,2024-09-01,4873.0
297,2024-10-01,4862.0
298,2024-11-01,5410.0


In [44]:
df = df.reset_index()

In [48]:
df = df.drop(columns=['Retail Avg.Price (Rupees per Quintal)'])

In [54]:
df.to_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Onion filled values.xlsx')


In [60]:
df = df.set_index('Date')

In [62]:
df

,W.sale Avg.Price(Rupees per Quintal)
Date,
2000-01-01,455.0
2000-02-01,358.0
2000-03-01,350.0
2000-04-01,367.0
2000-05-01,369.0
...,...
2024-08-01,4102.0
2024-09-01,4873.0
2024-10-01,4862.0


In [56]:
estimated_df = pd.read_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/1. Estimated Data.xlsx',index_col='Date')

In [58]:
estimated_df

,Temperature,Inflation_rate,Growth_rate,Log_MS
Date,,,,
1991-01-01,-1.6,16.091935,4.644682,14.764768
1991-02-01,0.4,15.428546,7.792836,14.783057
1991-03-01,1.4,13.559333,8.108108,14.793190
1991-04-01,-0.1,12.222223,0.811771,14.817024
1991-05-01,0.8,12.087911,-1.981179,14.836756
...,...,...,...,...
2022-08-01,-0.2,6.998158,-0.679758,19.165064
2022-09-01,0.1,7.414216,3.320463,19.166083
2022-10-01,-0.7,6.767372,-4.074074,19.176894


In [64]:
# Start with df as the base
new_df = df.copy()

# Align estimated_df to df's index and join the columns
# Only rows with matching indices will be added
estimated_aligned = estimated_df.loc[estimated_df.index.intersection(df.index)]

# Add the columns to new_df
new_df = new_df.join(estimated_aligned)

In [123]:
new_df

,Date,W.sale Avg.Price(Rupees per Quintal),Inflation_rate,Growth_rate,Log_MS
0,2000-01-01,455.0,2.619042,9.043591,16.208273
1,2000-02-01,358.0,3.614462,24.324022,16.223453
2,2000-03-01,350.0,4.830924,30.215964,16.235144
3,2000-04-01,367.0,5.542171,27.288814,16.261410
4,2000-05-01,369.0,5.011930,38.194843,16.267717
...,...,...,...,...,...
271,2022-08-01,1834.0,6.998158,-0.679758,19.165064
272,2022-09-01,1672.0,7.414216,3.320463,19.166083
273,2022-10-01,2290.0,6.767372,-4.074074,19.176894
274,2022-11-01,2136.0,5.878824,7.578125,19.181483


In [68]:
new_df = new_df.dropna()


In [72]:
new_df = new_df.reset_index()

In [74]:
zero_counts = (new_df == 0).sum()
print("Zero counts per column:")
print(zero_counts)

print("\nIndices of zero values per column:")
# Iterate through the columns and find indices of zero values
for column in new_df.columns:
    if (new_df[column] == 0).any():  # Check if the column has any zeros
        zero_indices = new_df[new_df[column] == 0].index.tolist()
        print(f"Zero values found in column '{column}' at indices: {zero_indices}")

Zero counts per column:
Date                                     0
W.sale Avg.Price(Rupees per Quintal)     0
Temperature                             14
Inflation_rate                           0
Growth_rate                              2
Log_MS                                   0
dtype: int64

Indices of zero values per column:
Zero values found in column 'Temperature' at indices: [11, 18, 19, 61, 75, 80, 84, 140, 152, 171, 182, 196, 210, 216]
Zero values found in column 'Growth_rate' at indices: [45, 242]


In [78]:
new_df = new_df.drop(columns=['Temperature'])


In [84]:
new_df.to_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Onion Joined.xlsx')


In [143]:
data = pd.read_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Onion Joined.xlsx',index_col='Date')

In [145]:
data

,Unnamed: 0,W.sale Avg.Price(Rupees per Quintal),Inflation_rate,Growth_rate,Log_MS
Date,,,,,
2000-01-01,0,455.0,2.619042,9.043591,16.208273
2000-02-01,1,358.0,3.614462,24.324022,16.223453
2000-03-01,2,350.0,4.830924,30.215964,16.235144
2000-04-01,3,367.0,5.542171,27.288814,16.261410
2000-05-01,4,369.0,5.011930,38.194843,16.267717
...,...,...,...,...,...
2022-08-01,271,1834.0,6.998158,-0.679758,19.165064
2022-09-01,272,1672.0,7.414216,3.320463,19.166083
2022-10-01,273,2290.0,6.767372,-4.074074,19.176894


In [147]:
data = data.drop(columns=['Unnamed: 0'])


In [149]:
print("Missing values:", data.isnull().sum())

Missing values: W.sale Avg.Price(Rupees per Quintal)    0
Inflation_rate                          0
Growth_rate                             0
Log_MS                                  0
dtype: int64


In [151]:
data = data.rename(columns={'W.sale Avg.Price(Rupees per Quintal)': 'W.sale Avg.Price'})

In [153]:
data

,W.sale Avg.Price,Inflation_rate,Growth_rate,Log_MS
Date,,,,
2000-01-01,455.0,2.619042,9.043591,16.208273
2000-02-01,358.0,3.614462,24.324022,16.223453
2000-03-01,350.0,4.830924,30.215964,16.235144
2000-04-01,367.0,5.542171,27.288814,16.261410
2000-05-01,369.0,5.011930,38.194843,16.267717
...,...,...,...,...
2022-08-01,1834.0,6.998158,-0.679758,19.165064
2022-09-01,1672.0,7.414216,3.320463,19.166083
2022-10-01,2290.0,6.767372,-4.074074,19.176894


## Feature Engineering for Time Series Modeling

In [156]:
# Add temporal features
data['Month'] = data.index.month
data['Year'] = data.index.year
data['Quarter'] = data.index.quarter

# Add lagged features
for col in ['W.sale Avg.Price','Inflation_rate','Growth_rate', 'Log_MS']:
    data[f'{col}_1M_Lag'] = data[col].shift(1)
    data[f'{col}_3M_Lag'] = data[col].shift(3)

# Add rolling statistics
data['Price_3M_Rolling_Mean'] = data['W.sale Avg.Price'].rolling(window=3).mean()
data['Price_6M_Rolling_Mean'] = data['W.sale Avg.Price'].rolling(window=6).mean()
data['Price_3M_Rolling_Std'] = data['W.sale Avg.Price'].rolling(window=3).std()
data['Price_6M_Rolling_Std'] = data['W.sale Avg.Price'].rolling(window=6).std()

# Create interaction features
data['Inflation_Growth_Interaction'] = data['Inflation_rate'] * data['Growth_rate']

# Add seasonal decomposition features
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(data['W.sale Avg.Price'], model='multiplicative', period=12)
data['seasonal_component'] = decomposition.seasonal

# Drop rows with NaN values
data_processed = data.dropna()

# Updated exogenous variables after feature engineering
exog_enhanced = data_processed.drop(['W.sale Avg.Price'], axis=1)
target_processed = data_processed['W.sale Avg.Price']


In [158]:
data_processed

,W.sale Avg.Price,Inflation_rate,Growth_rate,Log_MS,Month,Year,Quarter,W.sale Avg.Price_1M_Lag,W.sale Avg.Price_3M_Lag,Inflation_rate_1M_Lag,...,Growth_rate_1M_Lag,Growth_rate_3M_Lag,Log_MS_1M_Lag,Log_MS_3M_Lag,Price_3M_Rolling_Mean,Price_6M_Rolling_Mean,Price_3M_Rolling_Std,Price_6M_Rolling_Std,Inflation_Growth_Interaction,seasonal_component
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-01,401.0,5.238095,38.171534,16.281510,6,2000,2,369.0,350.0,5.011930,...,38.194843,30.215964,16.267717,16.235144,379.000000,383.333333,19.078784,39.174822,199.946120,0.763640
2000-07-01,486.0,4.952828,13.306099,16.281959,7,2000,3,401.0,367.0,5.238095,...,38.171534,27.288814,16.281510,16.261410,418.666667,388.500000,60.467622,50.828142,65.902813,0.920284
2000-08-01,460.0,3.990609,22.953398,16.288343,8,2000,3,486.0,369.0,4.952828,...,13.306099,38.194843,16.281959,16.267717,449.000000,405.500000,43.554563,55.435548,91.598023,1.104205
2000-09-01,512.0,3.496508,41.795593,16.303125,9,2000,3,460.0,401.0,3.990609,...,22.953398,38.171534,16.288343,16.281510,486.000000,432.500000,26.000000,62.054009,146.138613,1.209747
2000-10-01,596.0,2.746003,47.315842,16.319555,10,2000,4,512.0,486.0,3.496508,...,41.795593,13.306099,16.303125,16.281959,522.666667,470.666667,68.624583,81.185385,129.929446,1.361030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,1834.0,6.998158,-0.679758,19.165064,8,2022,3,1752.0,1271.0,6.707692,...,2.205323,19.721981,19.164157,19.147374,1733.666667,1630.666667,110.645078,338.348144,-4.757056,1.104205
2022-09-01,1672.0,7.414216,3.320463,19.166083,9,2022,3,1834.0,1615.0,6.998158,...,-0.679758,12.622150,19.165064,19.146480,1752.666667,1560.166667,81.002058,256.389872,24.618631,1.209747
2022-10-01,2290.0,6.767372,-4.074074,19.176894,10,2022,4,1672.0,1752.0,7.414216,...,3.320463,2.205323,19.166083,19.164157,1932.000000,1739.000000,320.443443,332.169836,-27.570773,1.361030


In [162]:
data_processed.to_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Dataset and Code files/Onion Processed data.xlsx')